In [ ]:
# Purpose: Generate LASSO-selected Vars at various r^2 cutoffs to compare with MI Features Selected
# Inputs: mean-imputed data with Homelessness Indicators added
# Outputs: Joined features for each r^2 cutoff specified, selected by outcome, aggregated without duplicates
# Machine: 4-core PC, ~10/12 hrs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
%matplotlib inline


In [ ]:
#data_all = pd.read_csv('data_mean_imputed.csv',index_col='challengeID')
data_all = pd.read_csv('../output/data_mean_imputed_Homeless_added.csv', index_col = 'challengeID')
outcomes = pd.read_csv('../data/train.csv',index_col='challengeID')
feature_names = pd.read_csv('../output/features_cleaned_stage2.csv')

In [ ]:
np.shape(data_all)

## Lasso Feature Selection

In [ ]:
for R2_TARGET in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    Alphas = np.logspace(-3,2,50).tolist()
    vars_OC = []

    for OUTCOME in outcomes.columns.values:
        y = outcomes[OUTCOME]
        y = y.dropna()
        x_all = data_all.loc[(np.in1d(list(data_all.index),list(y.index))),:]
        r_2 = []
        for L in Alphas:
            reg = linear_model.Lasso(alpha = L)
            reg.fit(x_all,y)
            r_2.append(reg.score(x_all,y))

        reg = linear_model.Lasso()
        path = reg.path(x_all,y, alphas = Alphas)
        n = [np.sum(path[1][:,n] != 0) for n in range(0,len(Alphas))]
        r_2.reverse()
        Alphas.reverse()

        temp = [abs(i-R2_TARGET) for i in r_2]

        Alpha_O = Alphas[temp.index(min(temp))]
        coeff = pd.DataFrame(path[1][:,temp.index(min(temp))],index = x_all.columns.values) 
        feature_index = coeff != 0
        features = x_all.loc[:,feature_index.iloc[:,0]]
        x_lars = features.columns.values

        vars_OC.extend(x_lars)


    vars_UNIQUE = data_all.loc[:,np.unique(vars_OC)]
    vars_UNIQUE.to_csv(('../output/LASSO_ALL/Lasso_'+str(R2_TARGET)+'_Vars.csv'))
    